In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
from scipy.optimize import fmin

In [2]:
#kernel = [0.5 * RBF(length_scale=5, length_scale_bounds=(1, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=20, length_scale_bounds=(1, 30)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=1, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)), 0.5 * RBF(length_scale=15, length_scale_bounds=(10, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5))]

In [3]:
kernel = 0.5 * (RBF(length_scale=5, length_scale_bounds=(1, 10)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)))

In [4]:
X_ = np.linspace(-10, 50, 60 + 1)
#xmin = np.linspace(-5, 5, 3)
wave = np.linspace(3000, 9000, 200 + 1)
save_path = '/home/joao/Documents/'

In [5]:
file = open("treinamento.txt")
lines = file.readlines()
file.close()

In [6]:
training_set = []

In [7]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [8]:
training_set = ['SN2012fr']

In [9]:
for j in range(0,len(training_set)):
    
    list_data = [[[],[]] for y in range(0,201)]
    
    sn_names=glob.glob("/home/joao/Documents/pesquisa/templates/data_ext/"+training_set[j]+"*"+".dat")
    
    
    
    for p in range(0,len(sn_names)):

        sn_names[p] = sn_names[p][49:]
    
    #print(sn_names)
    
    print(training_set[j])
    
    #print(sn_names)
    
    if len(sn_names) > 4:
        
        for i in range(0,len(sn_names)):

            #y = []
            z = []

            file = open(sn_names[i])
            lines = file.readlines()
            file.close()

            for line in lines:
                if line.startswith('#'): continue
                co=line.rstrip().replace('INDEF','Nan').split()

                #y.append(co[0])
                z.append(co[1])
        

            #y = np.array(y, dtype=float)
            z = np.array(z, dtype=float)
            #print(z)
            
            
            for l in range(0,len(z)):
                
                if z[l] < 10**(-64):
                    
                     z[l] = 0
            
            
            timetemp = lines[0]
            timetemp = timetemp[:-1]
            timetemp = float(timetemp[1:])
            
            for jj in range(0,len(z)):
                
                list_data[jj][0].append([timetemp])
                list_data[jj][1].append(z[jj])

            #print(i)
            
        Export = []
        Export_cov = []

        for jj in range(0,len(list_data)):
            
            X_mod = []
            
            Y_mean_mod = []
            
            count = 0
            
            for jjj in range(0,len(list_data[jj][1])):
                
                if list_data[jj][1][jjj] > 10**(-64):
                    
                    count += 1        
                    
            #print(count/len(list_data[jj][1]))
            
            if count > 4:
                
                
                
                for ii in range(0,len(list_data[jj][1])):
                    
                    if list_data[jj][1][ii] > 10**(-64):
                        
                        Y_mean_mod.append(list_data[jj][1][ii])
                        X_mod.append(list_data[jj][0][ii])
                    
                    

                #option = []
                #score = []  

                #print(list_data[jj][1])
                ff = max(Y_mean_mod)
                #print(ff)
                Y_mean_mod = Y_mean_mod/ff

                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(X_mod, Y_mean_mod)
                y_mean, y_cov = gp.predict(X_[:, np.newaxis], return_cov=True)        
                
                #print(y_cov)

                for l in range(0,len(y_mean)):

                    Export.append(y_mean[l]*ff)
                    Export_cov.append(np.sqrt(np.diag(y_cov))[l]*ff)


            else:

                y_meaneff = [0 for y in range(0,61)]
                #print(len(y_mean), len(X_))

                for l in range(0,len(y_meaneff)):

                    Export.append(y_meaneff[l])
                    Export_cov.append(0)
             

        table = []
        for l in range(0,len(Export)):
            #table.append((Export[l],training_set[j]))
            table.append((Export[l], Export_cov[l]))

        name_of_file = training_set[j]

        completeName = os.path.join(save_path, name_of_file+".dat")

        f = open(completeName, 'w')
        f.write(tabulate(table, tablefmt="plain"))
        f.close()


SN2012fr
[[ 4.81049655e-02  1.36286085e-02  1.16794632e-02 ...  7.46063479e-06
   2.54468909e-06 -1.35434090e-06]
 [ 1.36286085e-02  4.49153817e-02  1.08173976e-02 ...  9.49033876e-06
   4.79649015e-06  9.03340098e-07]
 [ 1.16794632e-02  1.08173976e-02  4.24434372e-02 ...  1.07322399e-05
   6.63329604e-06  3.05767888e-06]
 ...
 [ 7.46063479e-06  9.49033876e-06  1.07322399e-05 ...  4.88629302e-02
   1.59543850e-02  1.53786640e-02]
 [ 2.54468909e-06  4.79649015e-06  6.63329604e-06 ...  1.59543850e-02
   4.86299656e-02  1.57470330e-02]
 [-1.35434090e-06  9.03340098e-07  3.05767888e-06 ...  1.53786640e-02
   1.57470330e-02  4.84503212e-02]]
[[ 4.78171750e-02  1.38735295e-02  1.18228589e-02 ...  4.76964826e-06
   1.10861261e-09 -3.61778960e-06]
 [ 1.38735295e-02  4.44450794e-02  1.09137571e-02 ...  7.62544275e-06
   2.82577198e-06 -1.02366081e-06]
 [ 1.18228589e-02  1.09137571e-02  4.18534392e-02 ...  9.78385059e-06
   5.34810335e-06  1.59007605e-06]
 ...
 [ 4.76964826e-06  7.62544275e-06  

[[ 1.50637990e-02  5.67017765e-03  4.41550253e-03 ... -2.60933575e-06
   1.90378084e-06  5.33973790e-06]
 [ 5.67017765e-03  1.28838204e-02  3.87543919e-03 ... -3.55691858e-06
  -3.28213732e-07  2.28811409e-06]
 [ 4.41550253e-03  3.87543919e-03  1.14115341e-02 ... -3.90669448e-06
  -2.12571037e-06 -5.00477893e-07]
 ...
 [-2.60933575e-06 -3.55691858e-06 -3.90669448e-06 ...  1.34860820e-02
   5.25613875e-03  5.00320783e-03]
 [ 1.90378084e-06 -3.28213732e-07 -2.12571037e-06 ...  5.25613875e-03
   1.34048980e-02  5.20086588e-03]
 [ 5.33973790e-06  2.28811409e-06 -5.00477893e-07 ...  5.00320783e-03
   5.20086588e-03  1.33717820e-02]]
[[ 1.40214211e-02  5.35284359e-03  4.13835596e-03 ... -7.27273308e-07
   3.58347932e-06  6.70715532e-06]
 [ 5.35284359e-03  1.19042633e-02  3.61833879e-03 ... -2.22894223e-06
   9.86853453e-07  3.47761015e-06]
 [ 4.13835596e-03  3.61833879e-03  1.04884995e-02 ... -3.17214659e-06
  -1.24614059e-06  4.20481685e-07]
 ...
 [-7.27273308e-07 -2.22894223e-06 -3.1721465

[[ 6.65091620e-03  2.93857537e-03  2.03863554e-03 ...  2.60260748e-06
  -3.63661553e-07 -2.80249925e-06]
 [ 2.93857537e-03  5.02010868e-03  1.69106239e-03 ...  2.76179373e-06
   1.06079045e-06 -4.87539416e-07]
 [ 2.03863554e-03  1.69106239e-03  4.07107404e-03 ...  2.37549518e-06
   1.84195257e-06  1.17821887e-06]
 ...
 [ 2.60260748e-06  2.76179373e-06  2.37549518e-06 ...  4.85446797e-03
   2.08322716e-03  1.96598673e-03]
 [-3.63661553e-07  1.06079045e-06  1.84195257e-06 ...  2.08322716e-03
   4.85081516e-03  2.09319378e-03]
 [-2.80249925e-06 -4.87539416e-07  1.17821887e-06 ...  1.96598673e-03
   2.09319378e-03  4.86990221e-03]]
[[ 5.67923469e-03  2.51938083e-03  1.74290130e-03 ...  3.13668586e-06
   4.55058801e-07 -1.84962044e-06]
 [ 2.51938083e-03  4.26681640e-03  1.44169275e-03 ...  2.98469460e-06
   1.50720670e-06  9.03508145e-08]
 [ 1.74290130e-03  1.44169275e-03  3.44905079e-03 ...  2.34544720e-06
   1.94671890e-06  1.38413064e-06]
 ...
 [ 3.13668586e-06  2.98469460e-06  2.3454472

[[ 2.09184529e-03  1.00400582e-03  6.10953057e-04 ... -3.08923680e-06
  -2.84065620e-06 -2.24492020e-06]
 [ 1.00400582e-03  1.34728431e-03  4.77279464e-04 ... -2.13359908e-06
  -2.24714102e-06 -2.04477966e-06]
 [ 6.10953057e-04  4.77279464e-04  9.77345677e-04 ... -1.07561304e-06
  -1.36138006e-06 -1.42688540e-06]
 ...
 [-3.08923680e-06 -2.13359908e-06 -1.07561304e-06 ...  1.18714128e-03
   5.54793968e-04  5.20734256e-04]
 [-2.84065620e-06 -2.24714102e-06 -1.36138006e-06 ...  5.54793968e-04
   1.19769661e-03  5.63935081e-04]
 [-2.24492020e-06 -2.04477966e-06 -1.42688540e-06 ...  5.20734256e-04
   5.63935081e-04  1.20361521e-03]]
[[ 2.15174808e-03  1.03204361e-03  6.29205717e-04 ... -3.23864838e-06
  -3.03299912e-06 -2.44936631e-06]
 [ 1.03204361e-03  1.38908760e-03  4.91929616e-04 ... -2.21110685e-06
  -2.35642710e-06 -2.16735401e-06]
 [ 6.29205717e-04  4.91929616e-04  1.00932753e-03 ... -1.09473226e-06
  -1.39893172e-06 -1.47552027e-06]
 ...
 [-3.23864838e-06 -2.21110685e-06 -1.0947322

[[ 1.75166955e-03  8.52744758e-04  4.94181161e-04 ...  8.62536989e-08
   1.29827397e-06  2.18334393e-06]
 [ 8.52744758e-04  1.06248603e-03  3.78399805e-04 ... -5.73835834e-07
  -6.45354256e-08  3.91576403e-07]
 [ 4.94181161e-04  3.78399805e-04  7.38550527e-04 ... -7.55913064e-07
  -7.47678409e-07 -6.37259844e-07]
 ...
 [ 8.62536989e-08 -5.73835834e-07 -7.55913064e-07 ...  9.14058778e-04
   4.45218123e-04  4.18124829e-04]
 [ 1.29827397e-06 -6.45354256e-08 -7.47678409e-07 ...  4.45218123e-04
   9.29025046e-04  4.55644747e-04]
 [ 2.18334393e-06  3.91576403e-07 -6.37259844e-07 ...  4.18124829e-04
   4.55644747e-04  9.33063112e-04]]
[[ 1.54245466e-03  7.52359514e-04  4.32057007e-04 ...  7.19130021e-07
   1.92461015e-06  2.71893140e-06]
 [ 7.52359514e-04  9.25163785e-04  3.29628109e-04 ... -1.69755920e-07
   3.68137095e-07  7.90617782e-07]
 [ 4.32057007e-04  3.29628109e-04  6.37956645e-04 ... -5.73370378e-07
  -5.24332060e-07 -4.08438872e-07]
 ...
 [ 7.19130021e-07 -1.69755920e-07 -5.7337037

...

[[ 1.73371140e-03  8.45196259e-04  4.86629834e-04 ...  5.97558854e-07
   1.90973016e-06  2.79954632e-06]
 [ 8.45196259e-04  1.04320293e-03  3.71647488e-04 ... -3.01874087e-07
   2.74752314e-07  7.43958050e-07]
 [ 4.86629834e-04  3.71647488e-04  7.20999286e-04 ... -6.76134024e-07
  -6.34290743e-07 -5.09630526e-07]
 ...
 [ 5.97558854e-07 -3.01874087e-07 -6.76134024e-07 ...  8.95066369e-04
   4.38587933e-04  4.11991918e-04]
 [ 1.90973016e-06  2.74752314e-07 -6.34290743e-07 ...  4.38587933e-04
   9.10818819e-04  4.49356596e-04]
 [ 2.79954632e-06  7.43958050e-07 -5.09630526e-07 ...  4.11991918e-04
   4.49356596e-04  9.14715734e-04]]
[[ 1.86685320e-03  9.07065901e-04  5.29989614e-04 ... -5.58940729e-07
   5.69792755e-07  1.48037852e-06]
 [ 9.07065901e-04  1.14383822e-03  4.07162421e-04 ... -9.40948429e-07
  -5.01061797e-07 -4.68529849e-08]
 [ 5.29989614e-04  4.07162421e-04  8.00854592e-04 ... -8.85875233e-07
  -9.23300151e-07 -8.29250099e-07]
 ...
 [-5.58940729e-07 -9.40948429e-07 -8.8587523

[[ 1.62140791e-03  7.91538099e-04  3.54154714e-04 ... -1.03615254e-05
  -1.17838338e-05 -1.13120483e-05]
 [ 7.91538099e-04  7.35952897e-04  2.51318606e-04 ... -5.13978216e-06
  -5.88993264e-06 -5.69193601e-06]
 [ 3.54154714e-04  2.51318606e-04  4.10597545e-04 ... -1.07918460e-06
  -1.27007935e-06 -1.25546580e-06]
 ...
 [-1.03615254e-05 -5.13978216e-06 -1.07918460e-06 ...  8.39875546e-04
   6.58474687e-04  6.48133099e-04]
 [-1.17838338e-05 -5.88993264e-06 -1.27007935e-06 ...  6.58474687e-04
   1.00081703e-03  7.57425593e-04]
 [-1.13120483e-05 -5.69193601e-06 -1.25546580e-06 ...  6.48133099e-04
   7.57425593e-04  1.02310944e-03]]
[[ 1.64749928e-03  8.05758323e-04  3.67455338e-04 ... -1.08985748e-05
  -1.26217401e-05 -1.23117866e-05]
 [ 8.05758323e-04  7.62895635e-04  2.62012885e-04 ... -5.08068885e-06
  -5.92943067e-06 -5.82145159e-06]
 [ 3.67455338e-04  2.62012885e-04  4.32748146e-04 ... -8.22022152e-07
  -9.96334602e-07 -1.00859078e-06]
 ...
 [-1.08985748e-05 -5.08068885e-06 -8.2202215

[[ 1.98712042e-03  9.73062333e-04  5.46652418e-04 ...  3.08637320e-06
   4.99303393e-06  5.98237128e-06]
 [ 9.73062333e-04  1.16005384e-03  4.13448235e-04 ...  9.60175807e-07
   1.89540059e-06  2.45833632e-06]
 [ 5.46652418e-04  4.13448235e-04  7.84499829e-04 ... -3.63473617e-07
  -1.77404768e-07  1.23667768e-08]
 ...
 [ 3.08637320e-06  9.60175807e-07 -3.63473617e-07 ...  9.86463167e-04
   4.94965276e-04  4.65503474e-04]
 [ 4.99303393e-06  1.89540059e-06 -1.77404768e-07 ...  4.94965276e-04
   1.00892541e-03  5.09485582e-04]
 [ 5.98237128e-06  2.45833632e-06  1.23667768e-08 ...  4.65503474e-04
   5.09485582e-04  1.01305165e-03]]
[[ 2.41986854e-03  1.17728580e-03  6.84163253e-04 ... -1.75413516e-07
   1.42868526e-06  2.63923918e-06]
 [ 1.17728580e-03  1.47283173e-03  5.24458832e-04 ... -9.43321977e-07
  -2.84425693e-07  3.32877832e-07]
 [ 6.84163253e-04  5.24458832e-04  1.02630340e-03 ... -1.08255617e-06
  -1.09148977e-06 -9.48924217e-07]
 ...
 [-1.75413516e-07 -9.43321977e-07 -1.0825561

[[ 1.48533808e-03  7.21693477e-04  4.21683114e-04 ... -4.45397832e-07
   4.52480129e-07  1.17693173e-06]
 [ 7.21693477e-04  9.10093043e-04  3.23957864e-04 ... -7.48993961e-07
  -3.99119031e-07 -3.77732783e-08]
 [ 4.21683114e-04  3.23957864e-04  6.37205001e-04 ... -7.04912250e-07
  -7.34739738e-07 -6.59937570e-07]
 ...
 [-4.45397832e-07 -7.48993961e-07 -7.04912250e-07 ...  7.85749067e-04
   3.79900717e-04  3.56698226e-04]
 [ 4.52480129e-07 -3.99119031e-07 -7.34739738e-07 ...  3.79900717e-04
   7.97463129e-04  3.88282774e-04]
 [ 1.17693173e-06 -3.77732783e-08 -6.59937570e-07 ...  3.56698226e-04
   3.88282774e-04  8.01003871e-04]]
[[ 1.90698342e-03  9.19735772e-04  5.51723881e-04 ... -2.25372919e-06
  -1.69078437e-06 -9.72468829e-07]
 [ 9.19735772e-04  1.20685281e-03  4.28448347e-04 ... -1.74168636e-06
  -1.64999600e-06 -1.34719405e-06]
 [ 5.51723881e-04  4.28448347e-04  8.64654406e-04 ... -1.01900135e-06
  -1.20595544e-06 -1.20559930e-06]
 ...
 [-2.25372919e-06 -1.74168636e-06 -1.0190013

[[ 5.43752102e-03  2.52472919e-03  1.64543253e-03 ... -5.45348820e-06
  -8.37607114e-06 -9.81246149e-06]
 [ 2.52472919e-03  3.80670418e-03  1.32419178e-03 ... -2.44208195e-06
  -4.55434359e-06 -5.78154647e-06]
 [ 1.64543253e-03  1.32419178e-03  2.92853455e-03 ... -7.50617412e-08
  -1.24313834e-06 -2.08259658e-06]
 ...
 [-5.45348820e-06 -2.44208195e-06 -7.50617412e-08 ...  3.50211788e-03
   1.55821151e-03  1.46565186e-03]
 [-8.37607114e-06 -4.55434359e-06 -1.24313834e-06 ...  1.55821151e-03
   3.51045031e-03  1.57349945e-03]
 [-9.81246149e-06 -5.78154647e-06 -2.08259658e-06 ...  1.46565186e-03
   1.57349945e-03  3.52873239e-03]]
[[ 5.68274203e-03  2.63696402e-03  1.72028483e-03 ... -5.55976831e-06
  -8.64090632e-06 -1.01796439e-05]
 [ 2.63696402e-03  3.98312500e-03  1.38504559e-03 ... -2.45322625e-06
  -4.66926961e-06 -5.97065921e-06]
 [ 1.72028483e-03  1.38504559e-03  3.06682229e-03 ... -2.37165603e-08
  -1.24033821e-06 -2.12146797e-06]
 ...
 [-5.55976831e-06 -2.45322625e-06 -2.3716560

In [13]:
y_cov

array([[ 7.44688839e-03,  3.39286803e-03,  2.27338496e-03, ...,
        -1.85797967e-06, -6.29313122e-06, -9.28093236e-06],
       [ 3.39286803e-03,  5.38892469e-03,  1.85269224e-03, ...,
         3.99189261e-07, -2.45394144e-06, -4.57078020e-06],
       [ 2.27338496e-03,  1.85269224e-03,  4.24314260e-03, ...,
         1.76981330e-06,  4.92854385e-07, -6.48939560e-07],
       ...,
       [-1.85797967e-06,  3.99189261e-07,  1.76981330e-06, ...,
         5.05664668e-03,  2.20992428e-03,  2.08181762e-03],
       [-6.29313122e-06, -2.45394144e-06,  4.92854385e-07, ...,
         2.20992428e-03,  5.05965943e-03,  2.22619505e-03],
       [-9.28093236e-06, -4.57078020e-06, -6.48939560e-07, ...,
         2.08181762e-03,  2.22619505e-03,  5.08403674e-03]])